In [1]:
import tf_emddistance
import numpy as np
import tensorflow as tf
import random

In [8]:
# generate a series of point clouds sampled from a series of circle
def generate_circle_points(n_clouds,n_points,r_min,r_max):
    S = np.zeros([n_clouds,n_points,3])
    np.random.seed(291)
    for index_cloud in range(n_clouds):       
        r = np.random.uniform(r_min,r_max)
        for index_point in range(n_points):
            theta = np.random.uniform(0,2*np.pi)
            S[index_cloud][index_point][0:2] = [r*np.cos(theta),r*np.sin(theta)]
    return S

In [17]:
n_clouds = 100
n_points = 500
r_min = 1
r_max = 10

S = tf.placeholder(tf.float32,[None,n_points,3])
x = tf.Variable(tf.truncated_normal([1,n_points,3],mean=0.0,stddev=0.1), dtype = tf.float32)
X = tf.tile(x,[tf.shape(S)[0],1,1])
dist,_,_ = tf_emddistance.emd_distance(S,X)
model_loss = tf.reduce_mean(dist)*10000
avg_r = tf.reduce_mean(tf.norm(x,axis = 2))
train = tf.train.GradientDescentOptimizer(1e-3).minimize(model_loss)

iter_times = 300
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    S_circle = generate_circle_points(n_clouds,n_points,r_min,r_max)   
    for iters in range(iter_times):
        sess.run(train, {S: S_circle})
        loss,r = sess.run([model_loss,avg_r],{S: S_circle})
        if iters%10 == 0:
            print('iter: ', iters, '\tloss: ', loss, '\tavg_r: ', r)
        
    x_final = sess.run(x,{S: S_circle})
    x_final = np.reshape(x_final,[n_points,3])

iter:  0 	loss:  357610.8 	avg_r:  0.33978266
iter:  10 	loss:  200051.23 	avg_r:  2.105113
iter:  20 	loss:  129928.55 	avg_r:  3.289131
iter:  30 	loss:  98789.8 	avg_r:  4.077639
iter:  40 	loss:  84987.17 	avg_r:  4.6025643
iter:  50 	loss:  78845.625 	avg_r:  4.951979
iter:  60 	loss:  76128.28 	avg_r:  5.184564
iter:  70 	loss:  74909.625 	avg_r:  5.3393846
iter:  80 	loss:  74370.93 	avg_r:  5.4424434
iter:  90 	loss:  74129.55 	avg_r:  5.5110407
iter:  100 	loss:  74014.28 	avg_r:  5.556702
iter:  110 	loss:  73958.16 	avg_r:  5.5871058
iter:  120 	loss:  73946.15 	avg_r:  5.607348
iter:  130 	loss:  73936.125 	avg_r:  5.620813
iter:  140 	loss:  73926.2 	avg_r:  5.6297703
iter:  150 	loss:  73936.19 	avg_r:  5.6357436
iter:  160 	loss:  73931.22 	avg_r:  5.639717
iter:  170 	loss:  73918.04 	avg_r:  5.642364
iter:  180 	loss:  73921.14 	avg_r:  5.6441236
iter:  190 	loss:  73919.4 	avg_r:  5.645302
iter:  200 	loss:  73929.164 	avg_r:  5.6460743
iter:  210 	loss:  73927.234 	a

In [50]:
import pandas as pd
from pyntcloud import PyntCloud
red_color = np.tile(np.array([255,0,0]),[n_points,1])
grey_color = np.tile(np.array([255,255,255]),[n_clouds*n_points,1])
colors = (np.vstack([red_color,grey_color])).astype(np.uint8)
P = np.vstack([x_final,np.reshape(S_circle,[n_clouds*n_points,3])])
points = pd.DataFrame(P, columns=['x', 'y', 'z'])
points[['red', 'blue', 'green']] = pd.DataFrame(colors, index=points.index)
cloud = PyntCloud(points)
cloud.plot(point_size = 0.01,lines=[], line_color=[])

In [1]:
import numpy as np
import scipy
import scipy.spatial
import scipy.optimize
import pickle
import random

In [83]:
X1 = pickle.load(open('X1.pkl', 'rb'))
X2 = pickle.load(open('X2.pkl', 'rb'))
D = scipy.spatial.distance.cdist(X1,X2)

In [84]:
row_ind, col_ind = scipy.optimize.linear_sum_assignment(D)
cost = np.sum(D[row_ind,col_ind])
print(cost)

46.931869049959204


In [86]:
D = np.array([[15,40,45],[20,60,35],[20,40,25]])

In [97]:
[row_a,col_a] = linear_assignment(D)
cost = np.sum(D[row_a,col_a])
print(cost)

4C1:  [[1 0 0]
 [1 0 0]
 [1 0 0]]
4C2:  [[ 0 25 30]
 [ 0 40 15]
 [ 0 20  5]]
4C1:  [[1 0 0]
 [1 0 0]
 [1 0 1]]
4C2:  [[ 0 20 25]
 [ 0 35 10]
 [ 0 15  0]]
4C1:  [[1 0 0]
 [1 0 0]
 [1 0 1]]
4C2:  [[ 0 20 25]
 [ 0 35 10]
 [ 0 15  0]]
4C1:  [[1 0 0]
 [1 0 0]
 [1 1 1]]
4C2:  [[ 0  5 25]
 [ 0 20 10]
 [ 0  0  0]]
4C1:  [[1 1 0]
 [1 0 0]
 [0 1 1]]
4C2:  [[ 0  0 20]
 [ 0 15  5]
 [ 5  0  0]]
85


In [10]:
answer.shape

(500, 2)

In [96]:
"""
Solve the unique lowest-cost assignment problem using the
Hungarian algorithm (also known as Munkres algorithm).

"""
# Based on original code by Brain Clapper, adapted to NumPy by Gael Varoquaux.
# Heavily refactored by Lars Buitinck.
#
# TODO: a version of this algorithm has been incorporated in SciPy; use that
# when SciPy 0.17 is released.

# Copyright (c) 2008 Brian M. Clapper <bmc@clapper.org>, Gael Varoquaux
# Author: Brian M. Clapper, Gael Varoquaux
# LICENSE: BSD


def linear_assignment(X):
    indices = _hungarian(X)
    return indices


class _HungarianState(object):
    """State of one execution of the Hungarian algorithm.

    Parameters
    ----------
    cost_matrix : 2D matrix
        The cost matrix. Does not have to be square.
    """

    def __init__(self, cost_matrix):
        cost_matrix = np.atleast_2d(cost_matrix)

        self.C = cost_matrix.copy()
        n, m = self.C.shape
        self.row_uncovered = np.ones(n, dtype=np.bool)
        self.col_uncovered = np.ones(m, dtype=np.bool)
        self.Z0_r = 0
        self.Z0_c = 0
        self.path = np.zeros((n + m, 2), dtype=int)
        self.marked = np.zeros((n, m), dtype=int)

    def _find_prime_in_row(self, row):
        """
        Find the first prime element in the specified row. Returns
        the column index, or -1 if no starred element was found.
        """
        col = np.argmax(self.marked[row] == 2)
        if self.marked[row, col] != 2:
            col = -1
        return col

    def _clear_covers(self):
        """Clear all covered matrix cells"""
        self.row_uncovered[:] = True
        self.col_uncovered[:] = True


def _hungarian(cost_matrix):
    """The Hungarian algorithm.

    Calculate the Munkres solution to the classical assignment problem and
    return the indices for the lowest-cost pairings.

    Parameters
    ----------
    cost_matrix : 2D matrix
        The cost matrix. Does not have to be square.

    Returns
    -------
    indices : 2D array of indices
        The pairs of (row, col) indices in the original array giving
        the original ordering.
    """
    state = _HungarianState(cost_matrix)

    # No need to bother with assignments if one of the dimensions
    # of the cost matrix is zero-length.
    step = None if 0 in cost_matrix.shape else _step1

    while step is not None:
        step = step(state)
        
    # Look for the starred columns
    row_ind = np.where(state.marked == 1)[0]
    col_ind = np.where(state.marked == 1)[1]
    # results = np.array(np.where(state.marked == 1)).T
 
    return row_ind, col_ind


# Individual steps of the algorithm follow, as a state machine: they return
# the next step to be taken (function to be called), if any.

def _step1(state):
    """Steps 1 and 2 in the Wikipedia page."""

    # Step1: For each row of the matrix, find the smallest element and
    # subtract it from every element in its row.
    state.C -= state.C.min(axis=1)[:, np.newaxis]
    # Step2: Find a zero (Z) in the resulting matrix. If there is no
    # starred zero in its row or column, star Z. Repeat for each element
    # in the matrix.
    for i, j in zip(*np.where(state.C == 0)):
        if state.col_uncovered[j] and state.row_uncovered[i]:
            state.marked[i, j] = 1
            state.col_uncovered[j] = False
            state.row_uncovered[i] = False
    state._clear_covers()
    return _step3


def _step3(state):
    """
    Cover each column containing a starred zero. If n columns are covered,
    the starred zeros describe a complete set of unique assignments.
    In this case, Go to DONE, otherwise, Go to Step 4.
    """
    marked = (state.marked == 1)
    state.col_uncovered[np.any(marked, axis=0)] = False
    if marked.sum() < state.C.shape[0]:
        return _step4



def _step4(state):
    """
    Find a noncovered zero and prime it. If there is no starred zero
    in the row containing this primed zero, Go to Step 5. Otherwise,
    cover this row and uncover the column containing the starred
    zero. Continue in this manner until there are no uncovered zeros
    left. Save the smallest uncovered value and Go to Step 6.
    """
    # We convert to int as numpy operations are faster on int
    C = (state.C == 0).astype(np.int)
    print('4C1: ',C)
    print('4C2: ',state.C)
    covered_C = C * state.row_uncovered[:, np.newaxis]
    covered_C *= state.col_uncovered.astype(dtype=np.int, copy=False)
    n = state.C.shape[0]
    m = state.C.shape[1]
    while True:
        # Find an uncovered zero
        row, col = np.unravel_index(np.argmax(covered_C), (n, m))
        if covered_C[row, col] == 0:
            return _step6
        else:
            state.marked[row, col] = 2
            # Find the first starred element in the row
            star_col = np.argmax(state.marked[row] == 1)
            if not state.marked[row, star_col] == 1:
                # Could not find one
                state.Z0_r = row
                state.Z0_c = col
                return _step5
            else:
                col = star_col
                state.row_uncovered[row] = False
                state.col_uncovered[col] = True
                covered_C[:, col] = C[:, col] * (
                    state.row_uncovered.astype(dtype=np.int, copy=False))
                covered_C[row] = 0


def _step5(state):
    """
    Construct a series of alternating primed and starred zeros as follows.
    Let Z0 represent the uncovered primed zero found in Step 4.
    Let Z1 denote the starred zero in the column of Z0 (if any).
    Let Z2 denote the primed zero in the row of Z1 (there will always be one).
    Continue until the series terminates at a primed zero that has no starred
    zero in its column. Unstar each starred zero of the series, star each
    primed zero of the series, erase all primes and uncover every line in the
    matrix. Return to Step 3
    """
    count = 0
    path = state.path
    path[count, 0] = state.Z0_r
    path[count, 1] = state.Z0_c

    while True:
        # Find the first starred element in the col defined by
        # the path.
        row = np.argmax(state.marked[:, path[count, 1]] == 1)
        if not state.marked[row, path[count, 1]] == 1:
            # Could not find one
            break
        else:
            count += 1
            path[count, 0] = row
            path[count, 1] = path[count - 1, 1]

        # Find the first prime element in the row defined by the
        # first path step
        col = np.argmax(state.marked[path[count, 0]] == 2)
        if state.marked[row, col] != 2:
            col = -1
        count += 1
        path[count, 0] = path[count - 1, 0]
        path[count, 1] = col

    # Convert paths
    for i in range(count + 1):
        if state.marked[path[i, 0], path[i, 1]] == 1:
            state.marked[path[i, 0], path[i, 1]] = 0
        else:
            state.marked[path[i, 0], path[i, 1]] = 1

    state._clear_covers()
    # Erase all prime markings
    state.marked[state.marked == 2] = 0
    return _step3


def _step6(state):
    """
    Add the value found in Step 4 to every element of each covered row,
    and subtract it from every element of each uncovered column.
    Return to Step 4 without altering any stars, primes, or covered lines.
    """
    # the smallest uncovered value in the matrix
    if np.any(state.row_uncovered) and np.any(state.col_uncovered):
        minval = np.min(state.C[state.row_uncovered], axis=0)
        minval = np.min(minval[state.col_uncovered])
        state.C[np.logical_not(state.row_uncovered)] += minval
        state.C[:, state.col_uncovered] -= minval
    return _step4


In [41]:
a = np.random.random([4])
a[:, np.newaxis].shape

(4, 1)

In [9]:
X1 = pickle.load(open('S_teapot_euclidean.pkl', 'rb'))
X2 = pickle.load(open('S_violin_case_euclidean.pkl', 'rb'))

In [11]:
output = open('X2.pkl', 'wb')
pickle.dump(X2, output)
output.close()